In [1]:
import open3d as o3d
import numpy as np
import random
from collections import namedtuple

pcd = o3d.io.read_point_cloud(f'point_clouds/point_cloud80.ply')
o3d.visualization.draw_geometries_with_editing([pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
Point = namedtuple('Point', ['x', 'y', 'z'])


def check_collinearity(pos1, pos2, pos3):
    d1 = np.sqrt((pos2.x - pos3.x)**2 + (pos2.y - pos3.y)**2 + (pos2.z - pos3.z)**2)
    d2 = np.sqrt((pos1.x - pos2.x)**2 + (pos1.y - pos2.y)**2 + (pos1.z - pos2.z)**2)
    d3 = np.sqrt((pos3.x - pos1.x)**2 + (pos3.y - pos1.y)**2 + (pos3.z - pos1.z)**2)
    
    return abs(d1+d2-d3) < 1e-10 or abs(d2+d3-d1) < 1e-10 or abs(d3+d1-d2) < 1e-10


def dist(pos1, pos2, pos3, point):
    if check_collinearity(pos1, pos2, pos3) == True:
        raise Exception('points are colinear')
        
    a = pos1.y * (pos2.z - pos3.z) + pos2.y * (pos3.z - pos1.z) + pos3.y * (pos1.z - pos2.z)
    b = pos1.z * (pos2.x - pos3.x) + pos2.z * (pos3.x - pos1.x) + pos3.z * (pos1.x - pos2.x) 
    c = pos1.x * (pos2.y - pos3.y) + pos2.x * (pos3.y - pos1.y) + pos3.x * (pos1.y - pos2.y)
    d = -(pos1.x * (pos2.y * pos3.z - pos3.y * pos2.z) + 
          pos2.x * (pos3.y * pos1.z - pos1.y * pos3.z) + 
          pos3.x * (pos1.y * pos2.z - pos2.y * pos1.z))
    
    return abs(a * point.x + b * point.y + c * point.z + d) / np.sqrt(a * a + b * b + c * c)


def ransac(points, n_epochs=100, threshold=0.002, inliers_min = 100):
    best_n_inliers = 0
    best_points = None
    for _ in range(n_epochs):
        indx =  random.sample(range(len(points)), k=3)
        sample = points[indx]

        for p in points:
            if p in sample:
                continue
            
            d = dist(Point(*sample[0]), Point(*sample[1]), Point(*sample[2]), Point(*p))
            if d <= threshold:
                sample = np.append(sample, [p], axis=0)

        n_inliers = len(sample)
        if n_inliers < inliers_min:
            continue

        if best_n_inliers < n_inliers:
            best_points = sample
            best_n_inliers = n_inliers

    return best_points


pcd = o3d.io.read_point_cloud('point_clouds/point_cloud80.ply')
xyz = np.asarray(pcd.points)
points_on_plane = ransac(xyz)

pcd.paint_uniform_color([0,1,0])

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(points_on_plane)
pcd2.paint_uniform_color([1,0,0])

o3d.visualization.draw_geometries([pcd, pcd2])